# Data retrieval

Both IOER and NFDI provide standardized ways to access data. NFDI provides a [RESTful Application Programming Interface (API)](https://techdocs.gbif.org/en/openapi/), the IOER monitor allows to query spatial raster data as a [Web Coverage Service (WCS)](https://de.wikipedia.org/wiki/Web_Coverage_Service).

:::{tip}
Finding your way through the Python spatial package software ecosystem can be daunting. Before getting started, it is best to take some time to review the available packages and weigh the alternatives before jumping into writing code.
:::

For RESTful APIs, the go-to package in Python is the `requests' library. We have already used this library in the [previous section](201_example_introduction). For WCS queries in Python, it is necessary to check the available alternatives. For example, a quick search with one of the major web search engines may lead us to [GemGIS](https://gemgis.readthedocs.io/). This package provides a routine to query WCS, which we can find out by looking at its [docs](https://gemgis.readthedocs.io/en/stable/getting_started/tutorial/21_working_with_web_coverage_services.html). However, on a close look, we see that _GemGIS_ uses [OWSLib](https://github.com/geopython/OWSLib) under the hood. With OWSLib we can query the WCS directly in our Jupyter notebook. A web search can show some examples like the [ISIRC SoilGrids notebooks](https://www.isric.org/web-coverage-services-wcs) or the [NASAMAAP documentation](https://docs.maap-project.org/en/stable/technical_tutorials/access/edav_wcs_data.html).

# NFDI Occurrence Query

```{figure} https://techdocs.gbif.org/en/data-processing/_images/data-processing.png
:name: gbif-graphic

GBIF Data Processing Documentation (https://techdocs.gbif.org/en/data-processing/).
```

The RESTful service API from GBIF/NFDI is available at [techdocs.gbif.org](https://techdocs.gbif.org/). If you find API documentation like this, which is automatically build using [Swagger](https://swagger.io/), you are a big step further! Swagger based interactive API documentations allow to _test_ API endpoints before implementing any code.

We can test this for the occurence query by goijng to the following URL:

https://techdocs.gbif.org/en/openapi/v1/occurrence#/Searching%20occurrences/searchOccurrence

1. Click on **Try it out**
2. Enter `420` for the taxon-id of the English Sparrow (first line, `acceptedTaxonKey`). You can find the number 420 in the API response of the [previous section](201_example_introduction) (`json_data["results"][0]["taxonID"]`). 
3. Further down, click on **Execute**

# IÖR-Monitor WCS Query